# Introduction

![Logo](Images/z3logo.png)

Z3 is an efficient Satisfiability Modulo Theories (SMT) solver from Microsoft Research.

- Open Source
- Written in C/C++
- Default input format SMTLIB2
- Function interfaces for:
    - C/C++
    - .NET
    - Java
    - **Python**
    - Rust
    - OCAml
    - Julia
    - Smalltalk

## Installation

The python bindings can be installed via pip.

In [1]:
!pip install z3-solver


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


## OnePluZ3

In [2]:
from z3 import *

## Linear Programming - Factory Production

The new OnePluZ3 smartphone it about to go into production.  
Your location can produce the smartphone and some accessories. You are tasked with finding the optimal amount of each product to produce in order to maximize profit.

You have a limited amount of labor hours, machine time and raw material.
Maximize the profit given the following constraints.

You have 500 labor hours, 800 machine hours and 600 units of material.

|Name|ProductID|Profit|Labor Hours|Machine Time|Raw Materials|
|---|---|---|---|---|---|
|Phone Case|A|10|3|3|4|
|Phone Charger|B|30|5|3|2|
|Smarphone|C|50|4|5|6|


In [78]:
# Define Variables
A = Int('A') # Number of phone cases produced
B = Int('B') # Phone chargers
C = Int('C') # Smartphones

optimizer = Optimize() # Create a Z3 optimize environment

# Labor hours
optimizer.add(3*A + 5*B+ 4*C <= 500)

# Machine time
optimizer.add(3*A + 3*B + 5*C <= 800)

# Raw materials
optimizer.add(4*A + 2*B + 6*C <= 600)

# Non-negative restrictions
optimizer.add([A>=0, B>=0, C>=0])


# Define the objective function
profit = 10 * A + 30 * B + 50 * C

# Maximize the objective function
optimizer.maximize(profit)

# Check if the problem can be solved
if optimizer.check() == sat:
    model = optimizer.model()
    print("Optimal production plan:")
    print(f"Produce {model.evaluate(A)} cases.")
    print(f"Produce {model.evaluate(B)} chargers.")
    print(f"Produce {model.evaluate(C)} smartphones.")
    print(f"Maximum Profit: ${model.evaluate(profit)}")
else:
    print("No feasible production plan found.")


Optimal production plan:
Produce 0 cases.
Produce 27 chargers.
Produce 91 smartphones.
Maximum Profit: $5360


After showing this new plan to the sales team, they add a new constraint. If we produce more than 50 Smartphones, then we should produce at least enough phone cases for 30% of those. This is supposed to increase the number of sales.

In [79]:
optimizer.push()
optimizer.add(If(C > 50, A >= C * 0.3, True))

# Check if the problem can be solved
if optimizer.check() == sat:
    model = optimizer.model()
    print("Optimal production plan:")
    print(f"Produce {model.evaluate(A)} cases.")
    print(f"Produce {model.evaluate(B)} chargers.")
    print(f"Produce {model.evaluate(C)} smartphones.")
    print(f"Maximum Profit: ${model.evaluate(profit)}")
else:
    print("No feasible production plan found.")
optimizer.pop()


Optimal production plan:
Produce 23 cases.
Produce 25 chargers.
Produce 76 smartphones.
Maximum Profit: $4780


## Code Verification
The OS for the OnePluZ3 is written in C. While doing a code review of a coworkers code, you notice a strange C function.
```c
// Magic function
uint32_t f(int32_t v) {    
     
    int32_t const mask = v >> 31;
    uint32_t r = (v + mask) ^ mask;
    return r;
}
```
Unfortunately, your coworker left for their holiday and is unable to explain what this function is supposed to do.  
You try to figure out what it does by modeling it in python:

```
Speaker notes:
Notice how you can use the function normally in python, but you can also give a Z3 expression as parameter. This is the power of operator overloading.
```

In [80]:
def f(v):
    mask = v >> 31;
    r = (v + mask) ^ mask
    return r

In [81]:
print(f(0)) # => 0
print(f(1)) # => 1
print(f(5)) # => 5
print(f(100)) # => 100
print(f(1000)) # => 1000

0
1
5
100
1000


The function returns its input value? Something seems fishy, let's use Z3 to proof our theory.

In [82]:
x = BitVec("x",32)
y = f(x)
prove(x == y)

counterexample
[x = 4294967295]


Z3 interprets bitvectors as unsigned integers by default.
The counterexample is outside of the range of unsigned integers (max 2147483647).

In [85]:
import numpy as np
# Interpret as 32 bit unsigned, then reinterpret those bits as signed.
print(np.int32(np.uint32(2717999451)))

-1576967845


In [84]:
print(f(-1576967845)) # => 1576967845
print(f(-5)) # => 5
print(f(-100)) # => 100

1576967845
5
100


This function appears to return the absolute value of an integer. But you are still sceptical. How can you be sure that this is what this function does? And maybe even more importantly, that it won't break for a very specific input value?

Let's try to prove our new theory.

In [10]:
x = BitVec("x",32)
y = f(x)
prove(If(x >= 0,y == x, y == -x))
#prove(If(x >= 0,y == x, y == -x), show=True)

proved


```
Speaker notes:
In Z3 a proof is an exhaustive search without finding a counter example.
Switch commented lines just above. Show how the proof works.
```

## Dependency Chaos

After rolling out an update for *ZChat*, you notice that some services on that same server, including *MediaZync* have stopped working. After an investigation you determine that this update also installed the package *NetLib*.

Unfortunately, other services running on that server rely on a different package that is incompatible with *NetLib*. It looks like there might be a solution that works for both. How can you find this solution and verify that this won't cause further disruptions?

![title](Images/deps.png)

```
Speaker notes:
Obligatory joke about just using docker.
```

In [11]:
## Generate Constraint for Package dependency
def DependsOn(package, deps):
    return And( [ Implies(package, dep) for dep in deps ] )

# Generate constraint for conflicting packages
def Conflict(p1, p2):
    return Or(Not(p1), Not(p2))

In [12]:
a, b, c, d, e, netlib, g, zchat = Bools('a b c d e NetLib g ZChat')

s = Solver()

constraints = [
    DependsOn(a, [b, c]),
    DependsOn(b, [d]),
    DependsOn(c, [Or(d, e), Or(netlib, g)]),
    Conflict(d, e),
    Conflict(netlib, e),
    a,
    zchat
    ]

s.add(constraints)

if s.check() == sat:
    print(s.model())

[b = True,
 g = False,
 d = True,
 NetLib = True,
 c = True,
 a = True,
 e = False,
 ZChat = True]


## Optimizing dinner with Z3
To reward you for your great work, management decides to treat your team to a nice dinner. However, you have a very strict budget allocated for every minute detail of your meal. They even gave you an upper limit for much you can spend on appetizers!  
In an attempt to get every single cent from management, the following scene unfolds:
![title](https://imgs.xkcd.com/comics/np_complete.png)

[Source - xkcd 287](https://www.explainxkcd.com/wiki/index.php/287:_NP-Complete)

This problem is a variation of the knapsack problem and NP-complete.
You decide to help the waiter out and pull out your laptop:

In [13]:
appetizers = ["Mixed Fruit", "French Fries", "Side Salad", "Hot Wings", "Mozzarella Sticks", "Sampler Plate"]
prices = [2.15, 2.75, 3.35, 3.55, 4.20, 5.80]

a = IntVector('a', 6) # Int vector with 6 variables
s = Solver()

price_products = [price * var for price,var in zip(prices,a)]
total = sum(price_products)

#print(total)

s.add(total == 15.05)

for i in range(6):
    s.add(a[i] >= 0)

#s.add(a[0] != 7)
if s.check() == sat:
    m = s.model()
    for i in range(6):
        print(m.evaluate(a[i]),appetizers[i])
    print()


7 Mixed Fruit
0 French Fries
0 Side Salad
0 Hot Wings
0 Mozzarella Sticks
0 Sampler Plate



```
Speaker Notes:
a coworker pipes up, they don't want Mixed Fruit, add another restriction

Show total as Z3 interprets it
```

## Sudoku

During the dinner, the conversation shifts from work to leisure and someone mentions a challenging Sudoku puzzle they've been struggling with. You decide to demonstrate how versatile Z3 can be. It can even be used to solve logic puzzles.
```
Speaker Notes: Switch to sudoku solver notebook
```

## Z3 and Philosophy

You somehow land in a philosophical mood. #todo Come up with a better story.

Axioms:
1. All humans are mortal.
2. Socrates is a man.  

Conclusion :
- Socrates is mortal

In [32]:
Object = DeclareSort('Object') # uninterpreted sort

Human = Function('Human', Object, BoolSort())
Mortal = Function('Mortal', Object, BoolSort())

socrates = Const('socrates', Object)

x = Const('x', Object)

axioms = [
    ForAll([x], Implies(Human(x), Mortal(x))),
    Human(socrates)
]


s = Solver()
s.add(axioms)

print(s.check()) # sat => axioms are coherent

sat


In [29]:
# classical refutation
s.add(Not(Mortal(socrates))) # Socrates is not mortal

print(s.check()) # prints unsat so socrates is Mortal


unsat


Speaker Notes:  
Covers:
- Uninterpreted/free Sorts : Sorts with no a-priori interpretation
- uninterpreted/free functions
- Quantifiers
  
[Link](https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/advanced#uninterpreted-sorts)

## Simpsons

While watching TV ... #todo

### Entailment

Facts:
- Marge is parent of Lisa.
- Homer is parent of Bart.
- Homer is the father of Bart.

Rules:

![img](Images/father_parent.png)



![img](Images/parent_sibling.png)



In [87]:
Person = DeclareSort('Person') # uninterpreted sort

homer, marge, bart, lisa = Consts("homer marge bart lisa", Person)
hasFather = Function("hasFather", Person, Person)
parentOf = Function("parentOf", Person, Person, BoolSort())
siblings = Function("siblings", Person, Person, BoolSort())

x = Const('x', Person)
y = Const('y', Person)
z = Const('z', Person)

axioms = [
    parentOf(marge,lisa),
    parentOf(homer,lisa),
    hasFather(bart) == homer,
]
rules = [
    ForAll([x,y], Implies(hasFather(x) == y, parentOf(y,x))),
    ForAll([x,y,z], 
        Implies(
            And(parentOf(x,y), parentOf(x,z)),
            siblings(y,z)
    )),
]


s = Solver()
s.add(axioms)
s.add(rules)

#s.add(Not(siblings(bart,lisa)))

print(s.check())


sat
